## Baseball Prediction: 5b - Adding Starting Pitching Features
Now that we have raw game-level data for each pitcher, we can derive features based on the starting pitchers to help our prediction model for individual games.

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)



## Overall Plan of Attack
- For each starting pitcher we will load their raw data, create features for each game based on their previous performance, and then save the dataframe in a dictionary structure for easy lookup
- Then we can iterate through our game-level dataframe, add in the features for each starting pitcher, and use those to improve our model


In [25]:
df_gooden = pd.read_csv('pitching_data_goodd001.csv')

In [26]:
df_gooden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   at_vs        430 non-null    object 
 1   Opponent     430 non-null    object 
 2   League       430 non-null    object 
 3   GS           430 non-null    int64  
 4   CG           430 non-null    int64  
 5   SHO          430 non-null    int64  
 6   GF           430 non-null    int64  
 7   SV           430 non-null    int64  
 8   IP           430 non-null    float64
 9   H            430 non-null    int64  
 10  BFP          430 non-null    int64  
 11  HR           430 non-null    int64  
 12  R            430 non-null    int64  
 13  ER           430 non-null    int64  
 14  BB           430 non-null    int64  
 15  IB           430 non-null    int64  
 16  SO           430 non-null    int64  
 17  SH           430 non-null    int64  
 18  SF           430 non-null    int64  
 19  WP      

In [27]:
df_gooden.head(20)

,at_vs,Opponent,League,GS,CG,SHO,GF,SV,IP,H,BFP,HR,R,ER,BB,IB,SO,SH,SF,WP,HBP,BK,2B,3B,GDP,ROE,W,L,ERA,Date,dblhead_num
0,AT,HOU,N,1,0,0,0,0,5.0,3,20,0,1,1,2,0,5,0,0,0,0,0,0,0,0,0,1,0,1.80,4- 7-1984,NaN
1,AT,CHI,N,1,0,0,0,0,3.1,7,20,0,6,6,3,0,4,0,0,0,0,0,1,0,0,0,0,1,7.56,4-13-1984,NaN
2,VS,MON,N,1,0,0,0,0,5.0,5,26,0,4,0,4,0,7,0,0,0,1,0,0,0,0,1,0,0,4.72,4-19-1984,NaN
3,AT,MON,N,1,0,0,0,0,7.0,2,24,0,1,0,1,0,10,0,0,0,0,1,0,0,0,1,0,0,3.10,4-25-1984,NaN
4,VS,CHI,N,1,0,0,0,0,7.0,4,30,0,1,1,5,0,10,0,0,1,0,0,2,0,0,0,1,0,2.63,5- 1-1984,NaN
5,VS,HOU,N,1,0,0,0,0,2.1,6,15,0,8,8,2,0,4,0,0,0,0,1,0,0,0,0,0,1,4.85,5- 6-1984,NaN
6,AT,LA,N,1,1,1,0,0,9.0,4,32,0,0,0,2,0,11,0,0,0,0,1,0,0,0,0,1,0,3.72,5-11-1984,NaN
7,AT,SD,N,1,0,0,0,0,4.2,5,22,0,4,4,3,0,8,0,0,0,0,0,1,1,0,0,0,1,4.15,5-18-1984,NaN
8,VS,LA,N,1,0,0,0,0,8.0,3,30,1,1,1,3,0,14,0,0,0,0,0,1,0,0,0,1,0,3.68,5-25-1984,NaN
9,VS,STL,N,1,0,0,0,0,7.0,4,28,0,1,1,3,0,7,1,1,0,0,0,0,0,0,0,0,0,3.39,6- 1-1984,NaN


In [28]:
df_gooden['IP_real'] = df_gooden['IP_real'] = (df_gooden.IP - (df_gooden.IP % 1)) + (df_gooden.IP % 1) * (10/3)
df_gooden.head(10)

,at_vs,Opponent,League,GS,CG,SHO,GF,SV,IP,H,BFP,HR,R,ER,BB,IB,SO,SH,SF,WP,HBP,BK,2B,3B,GDP,ROE,W,L,ERA,Date,dblhead_num,IP_real
0,AT,HOU,N,1,0,0,0,0,5.0,3,20,0,1,1,2,0,5,0,0,0,0,0,0,0,0,0,1,0,1.80,4- 7-1984,NaN,5.000000
1,AT,CHI,N,1,0,0,0,0,3.1,7,20,0,6,6,3,0,4,0,0,0,0,0,1,0,0,0,0,1,7.56,4-13-1984,NaN,3.333333
2,VS,MON,N,1,0,0,0,0,5.0,5,26,0,4,0,4,0,7,0,0,0,1,0,0,0,0,1,0,0,4.72,4-19-1984,NaN,5.000000
3,AT,MON,N,1,0,0,0,0,7.0,2,24,0,1,0,1,0,10,0,0,0,0,1,0,0,0,1,0,0,3.10,4-25-1984,NaN,7.000000
4,VS,CHI,N,1,0,0,0,0,7.0,4,30,0,1,1,5,0,10,0,0,1,0,0,2,0,0,0,1,0,2.63,5- 1-1984,NaN,7.000000
5,VS,HOU,N,1,0,0,0,0,2.1,6,15,0,8,8,2,0,4,0,0,0,0,1,0,0,0,0,0,1,4.85,5- 6-1984,NaN,2.333333
6,AT,LA,N,1,1,1,0,0,9.0,4,32,0,0,0,2,0,11,0,0,0,0,1,0,0,0,0,1,0,3.72,5-11-1984,NaN,9.000000
7,AT,SD,N,1,0,0,0,0,4.2,5,22,0,4,4,3,0,8,0,0,0,0,0,1,1,0,0,0,1,4.15,5-18-1984,NaN,4.666667
8,VS,LA,N,1,0,0,0,0,8.0,3,30,1,1,1,3,0,14,0,0,0,0,0,1,0,0,0,1,0,3.68,5-25-1984,NaN,8.000000
9,VS,STL,N,1,0,0,0,0,7.0,4,28,0,1,1,3,0,7,1,1,0,0,0,0,0,0,0,0,0,3.39,6- 1-1984,NaN,7.000000


In [29]:
df_gooden.columns

Index(['at_vs', 'Opponent', 'League', 'GS', 'CG', 'SHO', 'GF', 'SV', 'IP', 'H',
       'BFP', 'HR', 'R', 'ER', 'BB', 'IB', 'SO', 'SH', 'SF', 'WP', 'HBP', 'BK',
       '2B', '3B', 'GDP', 'ROE', 'W', 'L', 'ERA', 'Date', 'dblhead_num',
       'IP_real'],
      dtype='object')

### Generating Features for a Starting Pitcher
- Want to creature features based on a lookback across $n$ games for each pitcher
- Need to aggregate statistics like innings pitched (IP), runs or earned runs given up, hits and walks given up, strikeouts, and so on
- We will also need to decide how to handle pitchers early in their career when they have relatively few games on which to base their performance.

In [30]:
def roll_column(df, col, winsize):
    # do the standard Pandas rolling calc
    t_col = df[col].rolling(winsize, closed='left').sum().to_numpy()
    
    # for the early columns, just do a rolling sum from the beginning
    t_col[:winsize] = np.concatenate(([0],df[col].iloc[:(winsize)].cumsum().to_numpy()[:-1]))

    return(t_col)

In [31]:
def load_and_process_pitch_df(p_id, filepath=''):
    fname = filepath+'pitching_data_'+p_id+'.csv'
    pitch_df = pd.read_csv(fname)
    
    # Convert date, fix dblhead_num to be 0,1,2
    pitch_df['Date'] = (pd.to_datetime(pitch_df.Date, format="%m-%d-%Y", errors="coerce")
                    .astype(str)
                    .str.replace('-', '')
                    .astype(int))
    pitch_df.dblhead_num.fillna(0, inplace=True)
    pitch_df['dblhead_num'] = pitch_df['dblhead_num'].astype(int)
    
    # Convert IP to proper mathematical format
    pitch_df['IP_real'] = (pitch_df.IP - (pitch_df.IP % 1)) + (pitch_df.IP % 1) * (10/3)
    
    cols_to_agg = ['IP_real', 'H','BFP', 'HR', 'R', 'ER', 'BB', 'IB', 'SO', 'SH', 'SF', 'WP', 'HBP', 'BK',
       '2B', '3B']
    winsizes = [10,35]
    for winsize in winsizes:
        for raw_col in cols_to_agg:
            new_colname = 'rollsum_'+raw_col+'_'+str(winsize)        
            pitch_df[new_colname] = roll_column(pitch_df, raw_col, winsize)

    
    er_per_ip_def = (5/9)
    h_bb_per_ip_def = 1.5
    h_bb_per_bf_def = .37
    so_per_bf_def = .2
    ip_per_game_def = 3
    bf_per_game_def = 12
    tb_bb_perc_def = .45
    for winsize in winsizes:
        hit_col = 'rollsum_H_'+str(winsize)
        bb_col = 'rollsum_BB_'+str(winsize)
        h_bb_col = 'H_BB_roll_'+str(winsize)
        double_col = 'rollsum_2B_'+str(winsize)
        triple_col = 'rollsum_3B_'+str(winsize)
        hr_col = 'rollsum_HR_'+str(winsize)
        xb_col = 'XB_roll_'+str(winsize)
        tb_col = 'TB_roll_'+str(winsize)
        so_col = 'rollsum_SO_'+str(winsize)
        so_mod_col = 'SO_mod_'+str(winsize)
        ip_col = 'rollsum_IP_real_'+str(winsize)
        ip_mod_col = 'IP_mod_'+str(winsize)
        er_col = 'rollsum_ER_'+str(winsize)
        er_mod_col = 'ER_mod_'+str(winsize)
        bf_col = 'rollsum_BFP_'+str(winsize)
        bf_mod_col = 'BF_mod_'+str(winsize)
        era_col = 'ERA_'+str(winsize)
        whip_col = 'WHIP_'+str(winsize)
        so_perc_col = 'SO_perc_'+str(winsize)
        h_bb_perc_col = 'H_BB_perc_'+str(winsize)
        h_bb_mod_col = 'H_BB_mod_'+str(winsize)
        h_bb_mod2_col = 'H_BB_mod2_'+str(winsize)
        tb_bb_mod_col = 'TB_BB_mod_'+str(winsize)
        tb_bb_perc_col = 'TB_BB_perc_'+str(winsize)
        pitch_df[h_bb_col] = pitch_df[hit_col]+pitch_df[bb_col]
        pitch_df[xb_col] = pitch_df[double_col]+2*pitch_df[triple_col]+2*pitch_df[hr_col]
        pitch_df[tb_col] = pitch_df[hit_col]+pitch_df[xb_col]
        pitch_df[ip_mod_col] = np.maximum(pitch_df[ip_col], winsize*ip_per_game_def)
        pitch_df[bf_mod_col] = np.maximum(pitch_df[bf_col], winsize*bf_per_game_def)
        pitch_df[er_mod_col] = pitch_df[er_col] + er_per_ip_def*(pitch_df[ip_mod_col]-pitch_df[ip_col])
        pitch_df[h_bb_mod_col] = pitch_df[h_bb_col] + h_bb_per_ip_def*(pitch_df[ip_mod_col]-pitch_df[ip_col])
        pitch_df[h_bb_mod2_col] = pitch_df[h_bb_col] + h_bb_per_bf_def*(pitch_df[bf_mod_col]-pitch_df[bf_col])
        pitch_df[so_mod_col] = pitch_df[so_col] + so_per_bf_def*(pitch_df[bf_mod_col]-pitch_df[bf_col])
        pitch_df[tb_bb_mod_col] = (pitch_df[tb_col] + pitch_df[bb_col])+ tb_bb_perc_def*(pitch_df[bf_mod_col]-pitch_df[bf_col])
        pitch_df[era_col] = (pitch_df[er_mod_col]/pitch_df[ip_mod_col])*9
        pitch_df[whip_col] = pitch_df[h_bb_mod_col]/pitch_df[ip_mod_col]
        pitch_df[so_perc_col] = pitch_df[so_mod_col]/pitch_df[bf_mod_col]
        pitch_df[tb_bb_perc_col] = pitch_df[tb_bb_mod_col]/pitch_df[bf_mod_col]
        pitch_df[h_bb_perc_col] = pitch_df[h_bb_mod2_col]/pitch_df[bf_mod_col]
    pitch_df['date_dblhead'] = (pitch_df['Date'].astype(str) + pitch_df['dblhead_num'].astype(str)).astype(int)
    pitch_df.set_index('date_dblhead', inplace=True)
    return(pitch_df)

In [32]:
df_gooden.head(50)

,at_vs,Opponent,League,GS,CG,SHO,GF,SV,IP,H,BFP,HR,R,ER,BB,IB,SO,SH,SF,WP,HBP,BK,2B,3B,GDP,ROE,W,L,ERA,Date,dblhead_num,IP_real
0,AT,HOU,N,1,0,0,0,0,5.0,3,20,0,1,1,2,0,5,0,0,0,0,0,0,0,0,0,1,0,1.80,4- 7-1984,NaN,5.000000
1,AT,CHI,N,1,0,0,0,0,3.1,7,20,0,6,6,3,0,4,0,0,0,0,0,1,0,0,0,0,1,7.56,4-13-1984,NaN,3.333333
2,VS,MON,N,1,0,0,0,0,5.0,5,26,0,4,0,4,0,7,0,0,0,1,0,0,0,0,1,0,0,4.72,4-19-1984,NaN,5.000000
3,AT,MON,N,1,0,0,0,0,7.0,2,24,0,1,0,1,0,10,0,0,0,0,1,0,0,0,1,0,0,3.10,4-25-1984,NaN,7.000000
4,VS,CHI,N,1,0,0,0,0,7.0,4,30,0,1,1,5,0,10,0,0,1,0,0,2,0,0,0,1,0,2.63,5- 1-1984,NaN,7.000000
5,VS,HOU,N,1,0,0,0,0,2.1,6,15,0,8,8,2,0,4,0,0,0,0,1,0,0,0,0,0,1,4.85,5- 6-1984,NaN,2.333333
6,AT,LA,N,1,1,1,0,0,9.0,4,32,0,0,0,2,0,11,0,0,0,0,1,0,0,0,0,1,0,3.72,5-11-1984,NaN,9.000000
7,AT,SD,N,1,0,0,0,0,4.2,5,22,0,4,4,3,0,8,0,0,0,0,0,1,1,0,0,0,1,4.15,5-18-1984,NaN,4.666667
8,VS,LA,N,1,0,0,0,0,8.0,3,30,1,1,1,3,0,14,0,0,0,0,0,1,0,0,0,1,0,3.68,5-25-1984,NaN,8.000000
9,VS,STL,N,1,0,0,0,0,7.0,4,28,0,1,1,3,0,7,1,1,0,0,0,0,0,0,0,0,0,3.39,6- 1-1984,NaN,7.000000


## Load in our game level data

In [33]:
df=pd.read_csv('df_bp3.csv')

/var/folders/4n/m6y9jtrx6m1_59wkdvlnxsm80000gn/T/ipykernel_14017/438026221.py:1: DtypeWarning: Columns (13,14,15,85,87,198) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('df_bp3.csv')


In [34]:
start_pitchers_h = df.pitcher_start_id_h.unique()
start_pitchers_v = df.pitcher_start_id_v.unique()
start_pitchers_all = np.union1d(start_pitchers_h, start_pitchers_v)


In [35]:
pitcher_data_dict = {}
for i, p_id in enumerate(start_pitchers_all):
    if i%100==0:
        print(i)
    pitcher_data_dict[p_id] = load_and_process_pitch_df(p_id)

0


/var/folders/4n/m6y9jtrx6m1_59wkdvlnxsm80000gn/T/ipykernel_14017/1442361754.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  pitch_df.dblhead_num.fillna(0, inplace=True)
/var/folders/4n/m6y9jtrx6m1_59wkdvlnxsm80000gn/T/ipykernel_14017/1442361754.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always

100


/var/folders/4n/m6y9jtrx6m1_59wkdvlnxsm80000gn/T/ipykernel_14017/1442361754.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  pitch_df.dblhead_num.fillna(0, inplace=True)
/var/folders/4n/m6y9jtrx6m1_59wkdvlnxsm80000gn/T/ipykernel_14017/1442361754.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always

200


...

/var/folders/4n/m6y9jtrx6m1_59wkdvlnxsm80000gn/T/ipykernel_14017/1442361754.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  pitch_df.dblhead_num.fillna(0, inplace=True)
/var/folders/4n/m6y9jtrx6m1_59wkdvlnxsm80000gn/T/ipykernel_14017/1442361754.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always

2800


/var/folders/4n/m6y9jtrx6m1_59wkdvlnxsm80000gn/T/ipykernel_14017/1442361754.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  pitch_df.dblhead_num.fillna(0, inplace=True)
/var/folders/4n/m6y9jtrx6m1_59wkdvlnxsm80000gn/T/ipykernel_14017/1442361754.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always

2900


/var/folders/4n/m6y9jtrx6m1_59wkdvlnxsm80000gn/T/ipykernel_14017/1442361754.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  pitch_df.dblhead_num.fillna(0, inplace=True)
/var/folders/4n/m6y9jtrx6m1_59wkdvlnxsm80000gn/T/ipykernel_14017/1442361754.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always

In [36]:
raw_cols_to_add = ['IP_real', 'rollsum_IP_real_10', 'rollsum_H_10',
       'rollsum_BFP_10', 'rollsum_HR_10', 'rollsum_R_10', 'rollsum_ER_10',
       'rollsum_BB_10', 'rollsum_IB_10', 'rollsum_SO_10', 'rollsum_SH_10',
       'rollsum_SF_10', 'rollsum_WP_10', 'rollsum_HBP_10',
       'rollsum_BK_10', 'rollsum_2B_10', 'rollsum_3B_10',
       'rollsum_IP_real_35', 'rollsum_H_35', 'rollsum_BFP_35',
       'rollsum_HR_35', 'rollsum_R_35', 'rollsum_ER_35', 'rollsum_BB_35',
       'rollsum_IB_35', 'rollsum_SO_35', 'rollsum_SH_35', 'rollsum_SF_35',
       'rollsum_WP_35', 'rollsum_HBP_35', 'rollsum_BK_35',
       'rollsum_2B_35', 'rollsum_3B_35', 'H_BB_roll_10', 'XB_roll_10',
       'TB_roll_10', 'IP_mod_10', 'BF_mod_10', 'ER_mod_10', 'H_BB_mod_10', 'H_BB_perc_10',
       'SO_mod_10', 'TB_BB_mod_10', 'ERA_10', 'WHIP_10', 'SO_perc_10',
       'TB_BB_perc_10', 'H_BB_roll_35', 'XB_roll_35', 'TB_roll_35',
       'IP_mod_35', 'BF_mod_35', 'ER_mod_35', 'H_BB_mod_35','H_BB_perc_35', 'SO_mod_35',
       'TB_BB_mod_35', 'ERA_35', 'WHIP_35', 'SO_perc_35', 'TB_BB_perc_35']
cols_to_add = [col+suff for col in raw_cols_to_add for suff in ['_h','_v']]

col_add_dict = {col:np.zeros(df.shape[0]) for col in cols_to_add}

In [37]:
for i in range(df.shape[0]):
    row = df.iloc[i,:]
    if i%1000==0:
        print(i)
    sp_id_v = row['pitcher_start_id_v']
    sp_id_h = row['pitcher_start_id_h']
    date_dblhead = row['date_dblhead']
    if sp_id_v in pitcher_data_dict.keys():
        curr_df = pitcher_data_dict[sp_id_v]
        if date_dblhead in curr_df.index:
            for col in raw_cols_to_add:
                col_add_dict[col+'_v'][i] = curr_df.loc[date_dblhead,col]
        else:
            print(f'no match for {sp_id_v} date {date_dblhead}')
    if sp_id_h in pitcher_data_dict.keys():
        curr_df = pitcher_data_dict[sp_id_h]
        if date_dblhead in curr_df.index:
            for col in raw_cols_to_add:
                col_add_dict[col+'_h'][i] = curr_df.loc[date_dblhead,col]
        else:
            print(f'no match for {sp_id_h} date {date_dblhead}')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000


In [38]:
for col in cols_to_add:
    df[col] = col_add_dict[col]

/var/folders/4n/m6y9jtrx6m1_59wkdvlnxsm80000gn/T/ipykernel_14017/4222469101.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = col_add_dict[col]
/var/folders/4n/m6y9jtrx6m1_59wkdvlnxsm80000gn/T/ipykernel_14017/4222469101.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = col_add_dict[col]
/var/folders/4n/m6y9jtrx6m1_59wkdvlnxsm80000gn/T/ipykernel_14017/4222469101.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

In [39]:
df.sample(5)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OBS_162_h,OBS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OBS_30_h,OBS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v,implied_prob_h,implied_prob_v,implied_prob_h_mid,over_under_line,over_under_result,IP_real_h,IP_real_v,rollsum_IP_real_10_h,rollsum_IP_real_10_v,rollsum_H_10_h,rollsum_H_10_v,rollsum_BFP_10_h,rollsum_BFP_10_v,rollsum_HR_10_h,rollsum_HR_10_v,rollsum_R_10_h,rollsum_R_10_v,rollsum_ER_10_h,rollsum_ER_10_v,rollsum_BB_10_h,rollsum_BB_10_v,rollsum_IB_10_h,rollsum_IB_10_v,rollsum_SO_10_h,rollsum_SO_10_v,rollsum_SH_10_h,rollsum_SH_10_v,rollsum_SF_10_h,rollsum_SF_10_v,rollsum_WP_10_h,rollsum_WP_10_v,rollsum_HBP_10_h,rollsum_HBP_10_v,rollsum_BK_10_h,rollsum_BK_10_v,rollsum_2B_10_h,rollsum_2B_10_v,rollsum_3B_10_h,rollsum_3B_10_v,rollsum_IP_real_35_h,rollsum_IP_real_35_v,rollsum_H_35_h,rollsum_H_35_v,rollsum_BFP_35_h,rollsum_BFP_35_v,rollsum_HR_35_h,rollsum_HR_35_v,rollsum_R_35_h,rollsum_R_35_v,rollsum_ER_35_h,rollsum_ER_35_v,rollsum_BB_35_h,rollsum_BB_35_v,rollsum_IB_35_h,rollsum_IB_35_v,rollsum_SO_35_h,rollsum_SO_35_v,rollsum_SH_35_h,rollsum_SH_35_v,rollsum_SF_35_h,rollsum_SF_35_v,rollsum_WP_35_h,rollsum_WP_35_v,rollsum_HBP_35_h,rollsum_HBP_35_v,rollsum_BK_35_h,rollsum_BK_35_v,rollsum_2B_35_h,rollsum_2B_35_v,rollsum_3B_35_h,rollsum_3B_35_v,H_BB_roll_10_h,H_BB_roll_10_v,XB_roll_10_h,XB_roll_10_v,TB_roll_10_h,TB_roll_10_v,IP_mod_10_h,IP_mod_10_v,BF_mod_10_h,BF_mod_10_v,ER_mod_10_h,ER_mod_10_v,H_BB_mod_10_h,H_BB_mod_10_v,H_BB_perc_10_h,H_BB_perc_10_v,SO_mod_10_h,SO_mod_10_v,TB_BB_mod_10_h,TB_BB_mod_10_v,ERA_10_h,ERA_10_v,WHIP_10_h,WHIP_10_v,SO_perc_10_h,SO_perc_10_v,TB_BB_perc_10_h,TB_BB_perc_10_v,H_BB_roll_35_h,H_BB_roll_35_v,XB_roll_35_h,XB_roll_35_v,TB_roll_35_h,TB_roll_35_v,IP_mod_35_h,IP_mod_35_v,BF_mod_35_h,BF_mod_35_v,ER_mod_35_h,ER_mod_35_v,H_BB_mod_35_h,H_BB_mod_35_v,H_BB_perc_35_h,H_BB_perc_35_v,SO_mod_35_h,SO_mod_35_v,TB_BB_mod_35_h,TB_BB_mod_35_v,ERA_35_h,ERA_35_v,WHIP_35_h,WHIP_35_v,SO_perc_35_h,SO_perc_35_v,TB_BB_perc_35_h,TB_BB_perc_35_v
29406,19940514,0,Sat,PIT,NL,34,PHI,NL,36,2,3,51,N,NaN,NaN,NaN,PHI12,39916.0,128,010001000,00200100x,31,8,1,1,0,2,0,1,0,4,0,4,0,0,2,0

In [40]:
df.shape

(92946, 321)

In [41]:
(df.IP_mod_10_h==0).sum()

0

In [42]:
(df.IP_mod_10_v==0).sum()

0

In [43]:
df.loc[df.IP_mod_10_v==0]

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OBS_162_h,OBS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OBS_30_h,OBS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v,implied_prob_h,implied_prob_v,implied_prob_h_mid,over_under_line,over_under_result,IP_real_h,IP_real_v,rollsum_IP_real_10_h,rollsum_IP_real_10_v,rollsum_H_10_h,rollsum_H_10_v,rollsum_BFP_10_h,rollsum_BFP_10_v,rollsum_HR_10_h,rollsum_HR_10_v,rollsum_R_10_h,rollsum_R_10_v,rollsum_ER_10_h,rollsum_ER_10_v,rollsum_BB_10_h,rollsum_BB_10_v,rollsum_IB_10_h,rollsum_IB_10_v,rollsum_SO_10_h,rollsum_SO_10_v,rollsum_SH_10_h,rollsum_SH_10_v,rollsum_SF_10_h,rollsum_SF_10_v,rollsum_WP_10_h,rollsum_WP_10_v,rollsum_HBP_10_h,rollsum_HBP_10_v,rollsum_BK_10_h,rollsum_BK_10_v,rollsum_2B_10_h,rollsum_2B_10_v,rollsum_3B_10_h,rollsum_3B_10_v,rollsum_IP_real_35_h,rollsum_IP_real_35_v,rollsum_H_35_h,rollsum_H_35_v,rollsum_BFP_35_h,rollsum_BFP_35_v,rollsum_HR_35_h,rollsum_HR_35_v,rollsum_R_35_h,rollsum_R_35_v,rollsum_ER_35_h,rollsum_ER_35_v,rollsum_BB_35_h,rollsum_BB_35_v,rollsum_IB_35_h,rollsum_IB_35_v,rollsum_SO_35_h,rollsum_SO_35_v,rollsum_SH_35_h,rollsum_SH_35_v,rollsum_SF_35_h,rollsum_SF_35_v,rollsum_WP_35_h,rollsum_WP_35_v,rollsum_HBP_35_h,rollsum_HBP_35_v,rollsum_BK_35_h,rollsum_BK_35_v,rollsum_2B_35_h,rollsum_2B_35_v,rollsum_3B_35_h,rollsum_3B_35_v,H_BB_roll_10_h,H_BB_roll_10_v,XB_roll_10_h,XB_roll_10_v,TB_roll_10_h,TB_roll_10_v,IP_mod_10_h,IP_mod_10_v,BF_mod_10_h,BF_mod_10_v,ER_mod_10_h,ER_mod_10_v,H_BB_mod_10_h,H_BB_mod_10_v,H_BB_perc_10_h,H_BB_perc_10_v,SO_mod_10_h,SO_mod_10_v,TB_BB_mod_10_h,TB_BB_mod_10_v,ERA_10_h,ERA_10_v,WHIP_10_h,WHIP_10_v,SO_perc_10_h,SO_perc_10_v,TB_BB_perc_10_h,TB_BB_perc_10_v,H_BB_roll_35_h,H_BB_roll_35_v,XB_roll_35_h,XB_roll_35_v,TB_roll_35_h,TB_roll_35_v,IP_mod_35_h,IP_mod_35_v,BF_mod_35_h,BF_mod_35_v,ER_mod_35_h,ER_mod_35_v,H_BB_mod_35_h,H_BB_mod_35_v,H_BB_perc_35_h,H_BB_perc_35_v,SO_mod_35_h,SO_mod_35_v,TB_BB_mod_35_h,TB_BB_mod_35_v,ERA_35_h,ERA_35_v,WHIP_35_h,WHIP_35_v,SO_perc_35_h,SO_perc_35_v,TB_BB_perc_35_h,TB_BB_perc_35_v


In [44]:
df.loc[df.IP_mod_10_h==0]

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OBS_162_h,OBS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OBS_30_h,OBS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v,implied_prob_h,implied_prob_v,implied_prob_h_mid,over_under_line,over_under_result,IP_real_h,IP_real_v,rollsum_IP_real_10_h,rollsum_IP_real_10_v,rollsum_H_10_h,rollsum_H_10_v,rollsum_BFP_10_h,rollsum_BFP_10_v,rollsum_HR_10_h,rollsum_HR_10_v,rollsum_R_10_h,rollsum_R_10_v,rollsum_ER_10_h,rollsum_ER_10_v,rollsum_BB_10_h,rollsum_BB_10_v,rollsum_IB_10_h,rollsum_IB_10_v,rollsum_SO_10_h,rollsum_SO_10_v,rollsum_SH_10_h,rollsum_SH_10_v,rollsum_SF_10_h,rollsum_SF_10_v,rollsum_WP_10_h,rollsum_WP_10_v,rollsum_HBP_10_h,rollsum_HBP_10_v,rollsum_BK_10_h,rollsum_BK_10_v,rollsum_2B_10_h,rollsum_2B_10_v,rollsum_3B_10_h,rollsum_3B_10_v,rollsum_IP_real_35_h,rollsum_IP_real_35_v,rollsum_H_35_h,rollsum_H_35_v,rollsum_BFP_35_h,rollsum_BFP_35_v,rollsum_HR_35_h,rollsum_HR_35_v,rollsum_R_35_h,rollsum_R_35_v,rollsum_ER_35_h,rollsum_ER_35_v,rollsum_BB_35_h,rollsum_BB_35_v,rollsum_IB_35_h,rollsum_IB_35_v,rollsum_SO_35_h,rollsum_SO_35_v,rollsum_SH_35_h,rollsum_SH_35_v,rollsum_SF_35_h,rollsum_SF_35_v,rollsum_WP_35_h,rollsum_WP_35_v,rollsum_HBP_35_h,rollsum_HBP_35_v,rollsum_BK_35_h,rollsum_BK_35_v,rollsum_2B_35_h,rollsum_2B_35_v,rollsum_3B_35_h,rollsum_3B_35_v,H_BB_roll_10_h,H_BB_roll_10_v,XB_roll_10_h,XB_roll_10_v,TB_roll_10_h,TB_roll_10_v,IP_mod_10_h,IP_mod_10_v,BF_mod_10_h,BF_mod_10_v,ER_mod_10_h,ER_mod_10_v,H_BB_mod_10_h,H_BB_mod_10_v,H_BB_perc_10_h,H_BB_perc_10_v,SO_mod_10_h,SO_mod_10_v,TB_BB_mod_10_h,TB_BB_mod_10_v,ERA_10_h,ERA_10_v,WHIP_10_h,WHIP_10_v,SO_perc_10_h,SO_perc_10_v,TB_BB_perc_10_h,TB_BB_perc_10_v,H_BB_roll_35_h,H_BB_roll_35_v,XB_roll_35_h,XB_roll_35_v,TB_roll_35_h,TB_roll_35_v,IP_mod_35_h,IP_mod_35_v,BF_mod_35_h,BF_mod_35_v,ER_mod_35_h,ER_mod_35_v,H_BB_mod_35_h,H_BB_mod_35_v,H_BB_perc_35_h,H_BB_perc_35_v,SO_mod_35_h,SO_mod_35_v,TB_BB_mod_35_h,TB_BB_mod_35_v,ERA_35_h,ERA_35_v,WHIP_35_h,WHIP_35_v,SO_perc_35_h,SO_perc_35_v,TB_BB_perc_35_h,TB_BB_perc_35_v


In [45]:
df.drop(df.index[df.IP_mod_10_v==0],inplace=True)
df.drop(df.index[df.IP_mod_10_h==0],inplace=True)
df.shape

(92946, 321)

In [46]:
df.reset_index(drop=True, inplace=True)

In [47]:
df.to_csv('df_bp5.csv', index=False)